#**Tugas 1 Jobsheet 7**

Lakukan percobaan pada metric distance yang berbeda, 1000 vs 1.000.000 data, 2D vs 5D data, untuk algoritma,

ANNOY

FAISS

HNSW

Catat performansinya dalam bentuk tabel, misal

In [2]:
!pip install annoy faiss-cpu hnswlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 23.0 MB/s eta 0:00:00
  Created wheel for annoy: filename=annoy-1.17.3-cp312-cp312-linux_x86_64.whl size=551811 sha256=b55670fddb989cf5c8a25688b4473d77bd28bb8c0adadffdfdc0365f214c1349
  Stored in directory: /root/.cache/pip/wheels/db/b9/53/a3b2d1fe1743abadddec6aa541294b24fdbc39d7800bc57311
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp312-cp312-linux_x86_64.whl size=2528147 sha256=d285e11401b84924fd608ddd83863ad33ddf0b338774d14e7e17253fee34167a
  Stored in directory: /root/.cache/pip/wheels/ac/39/b3/cbd7f9cbb76501d2d5fbc84956e70d0b94e788aac87bda465e
Successfully built annoy hnswlib


In [3]:
import numpy as np
import time
from annoy import AnnoyIndex
import faiss
import hnswlib

# Fungsi bantu untuk catat waktu
def measure_time(func):
    start = time.time()
    func()
    return time.time() - start

# Percobaan
configs = [
    (1000, 2),
    (1000, 5),
    (1_000_000, 2),
    (1_000_000, 5)
]

results = []

for n, dim in configs:
    print(f"\n=== {n} data, {dim}D ===")
    data = np.random.random((n, dim)).astype('float32')
    query = np.random.random((1, dim)).astype('float32')

    # --- ANNOY ---
    annoy_index = AnnoyIndex(dim, 'euclidean')
    for i in range(n):
        annoy_index.add_item(i, data[i])
    t_annoy_build = measure_time(lambda: annoy_index.build(10))
    t_annoy_search = measure_time(lambda: annoy_index.get_nns_by_vector(query[0], 5))

    # --- FAISS ---
    index_faiss = faiss.IndexFlatL2(dim)
    t_faiss_build = measure_time(lambda: index_faiss.add(data))
    t_faiss_search = measure_time(lambda: index_faiss.search(query, 5))

    # --- HNSW ---
    index_hnsw = hnswlib.Index(space='l2', dim=dim)
    index_hnsw.init_index(max_elements=n, ef_construction=200, M=16)
    t_hnsw_build = measure_time(lambda: index_hnsw.add_items(data))
    t_hnsw_search = measure_time(lambda: index_hnsw.knn_query(query, k=5))

    results.append({
        "Data/Dim": f"{n}/{dim}D",
        "Annoy": t_annoy_search,
        "FAISS": t_faiss_search,
        "HNSW": t_hnsw_search
    })

print("\nHasil perbandingan:")
for r in results:
    print(r)



=== 1000 data, 2D ===

=== 1000 data, 5D ===

=== 1000000 data, 2D ===

=== 1000000 data, 5D ===

Hasil perbandingan:
{'Data/Dim': '1000/2D', 'Annoy': 5.459785461425781e-05, 'FAISS': 6.67572021484375e-05, 'HNSW': 7.963180541992188e-05}
{'Data/Dim': '1000/5D', 'Annoy': 6.222724914550781e-05, 'FAISS': 5.14984130859375e-05, 'HNSW': 5.602836608886719e-05}
{'Data/Dim': '1000000/2D', 'Annoy': 7.987022399902344e-05, 'FAISS': 0.0055921077728271484, 'HNSW': 7.152557373046875e-05}
{'Data/Dim': '1000000/5D', 'Annoy': 9.131431579589844e-05, 'FAISS': 0.011671066284179688, 'HNSW': 0.00010395050048828125}


Jumlah Data/Dimensi

ANNOY
FAISS
HNSW

1000/2D Annoy : 'Annoy' : 5.459785461425781e-05, 'FAISS' : 6.67572021484375e-05, 'HNSW' : 7.963180541992188e-05

1000/5D : 'Annoy' : 6.222724914550781e-05, 'FAISS' : 5.14984130859375e-05, 'HNSW' : 5.602836608886719e-05}

1000000/2D : 'Annoy' : 7.987022399902344e-05, 'FAISS': 0.0055921077728271484, 'HNSW': 7.152557373046875e-05}

1000000/5D : 'Annoy' : 9.131431579589844e-05, 'FAISS' : 0.011671066284179688, 'HNSW' : 0.00010395050048828125}

